### Description

- Парсер пульса с прокруткой (для сбора начальной базы данных с помощью selenium)
- Преобразование даннных в pandas.dataframe
- В конце первые попытки подвязать gpt

### Libraries

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import emoji
import re
from datetime import datetime, timedelta
import json
import pandas as pd
from openai import OpenAI
from settings import AUTHORS, BASE_URL, tickers, DS_API_KEY, QUESTION_CONTEXT
import numpy as np
import pandas as pd
import json

### Parser

In [17]:
def get_question_context(file_path = QUESTION_CONTEXT):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            file_content = file.read()
            return file_content
    except FileNotFoundError:
        print(f"The file {file_path} does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [18]:
class Article:
    def __init__(self, text):
        self.text = self.clean_text(text)
    
    # def predict(self, query_text: str):
    #     question_context = get_question_context()
    #     client = OpenAI(api_key=DS_API_KEY, base_url="https://api.deepseek.com/v1")
    #     response = client.chat.completions.create(
    #         model="deepseek-chat",
    #         messages=[
    #             {"role": "user", "content": question_context},
    #             {"role": "user", "content": query_text},
    #             ]
    #         )
    #     answer = response.choices[0].message.content
    #     answer = answer.strip().upper()
    #     prediction = 0
    #     ticker = 0
    #     if "DOWN" in answer:
    #         prediction = -1
    #         answer = answer.replace("DOWN", "")
    #     elif "UP" in answer:
    #         prediction = 1
    #         answer = answer.replace("UP", "")
    #     else:
    #         return 0, 0
    #     if len(answer) < 20:
    #         ticker = re.sub(r'[+\-=:/]', '', answer)             
    #     return ticker, prediction


    def clean_text(self, text):
        emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F700-\U0001F77F"  # alchemical symbols
                                u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                                u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                                u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                                u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                                u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                                u"\U00002702-\U000027B0"  # Dingbats
                                u"\U000024C2-\U0001F251" 
                                "]+", flags=re.UNICODE)
        clean_text = emoji_pattern.sub(r'', text)
        return clean_text

    def fix_dates(self, date_string):
        if 'Сегодня' in date_string:
            result = datetime.combine(datetime.now().date(), 
                                      (datetime.combine(datetime.today(), datetime.strptime(date_string.split(" ")[-1], '%H:%M').time()) - timedelta(hours=3)).time()).strftime("%Y-%m-%d %H:%M")
        elif 'Вчера' in date_string:
            result = datetime.combine(datetime.now().date() - timedelta(days=1), 
                                      (datetime.combine(datetime.today(), datetime.strptime(date_string.split(" ")[-1], '%H:%M').time()) - timedelta(hours=3)).time()).strftime("%Y-%m-%d %H:%M")
        else:
            result_list = date_string.split(" ")
            result_list[1] = self.month_dict[result_list[1]]
            result_list.pop(3)

            result_list[-1]=str((int(result_list[-1].split(':')[0])+21)%24) + ":" + result_list[-1].split(':')[1]
            date_str = ' '.join(result_list)
            # result = date_str
            date_format = '%d %m %Y %H:%M'
            result = datetime.strptime(date_str, date_format).strftime("%Y-%m-%d %H:%M")

        return result


### Run and save

In [19]:
def get_base_url(name, start= BASE_URL):
    return start+name

def get_full_data(url):
    print("get_full_data")

    driver = webdriver.Chrome()
    driver.get(url)
    print("webdriver opened")

    try:
        for i in range(7):  
            print('start', i, end = ", ")
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 2000);")

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        data = soup.find_all('div', {'data-qa-tag':'PulsePost'})
        articles = []
        print(f"Загружено {len(data)} постов")

        for post in data:
            text = post.find_all('div', {'class':'pulse-profilepage__fGGBmY'})[0].get_text()

            article = Article(text)
            articles.append(article.text)

    finally:
        driver.quit()   
        
    return articles

In [20]:
AUTHORS

['Falcon', 'Mistika911', 'Dogtor']

In [13]:
final_list_texts = []
for author in AUTHORS:
    print(author, end = " ")
    url = get_base_url(author)
    author_texts = get_full_data(url)
    final_list_texts+=author_texts
# print(final_list_texts)
filename = './data/data_posts.json'
with open(filename, "w", encoding="utf-8") as json_file:
    json.dump(final_list_texts, json_file, ensure_ascii=False, indent=4)


Falcon get_full_data
webdriver opened
start 0, start 1, start 2, start 3, start 4, start 5, start 6, Загружено 210 постов
Mistika911 get_full_data
webdriver opened
start 0, start 1, start 2, start 3, start 4, start 5, start 6, Загружено 210 постов
Dogtor get_full_data
webdriver opened
start 0, start 1, start 2, start 3, start 4, start 5, start 6, Загружено 180 постов


In [21]:
file_content = "Твоя задача на основании анализа предложенного текста сделать выводы о дальнейшем движении цены на акцию. Ответ представь в формате числа от 0 до 1, где 0 - цена точно пойдет вниз, а 1 - цена точно пойдет вверх. Ты будешь оштрафован, если в ответе будет что-то кроме одного числа. \nВот сам текст: \n"


In [22]:
def predict(row: str):
    x= row
    client = OpenAI(api_key=DS_API_KEY, base_url="https://api.deepseek.com")
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": file_content},
            {"role": "user", "content": x},
            {"role": "user", "content": 'Напиши краткий ответ в одну строчку. Только название одной компании и направлении цены, больше ничего, два слова через пробел без пояснений'}
            ]
        )
    answer = response.choices[0].message.content
    answer = answer.strip().upper()
    print(answer) 
    return answer



In [23]:
filename = './data/data_posts.json'

with open(filename, 'r', encoding='utf-8') as file:
    json_content = file.read()
    data = json.loads(json_content)
    print(predict(data[0]))

APIStatusError: Error code: 402 - {'detail': 'Insufficient Balance'}

### Create DataFrame

In [4]:
df = pd.read_json("data_posts.json")
df = df.explode('articles')
df.reset_index(inplace=True, drop=True)

df['date'] = df['articles'].apply(lambda x: x.pop('date'))
df['text'] = df['articles'].apply(lambda x: x.pop('text'))
df.drop(columns=['articles'], inplace=True)

### GPT

In [5]:
file_path = 'question.txt'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")
except Exception as e:
    print(f"An error occurred: {e}")

In [7]:
def predict(row: str):
    x= row['text']
    client = OpenAI(api_key=DS_API_KEY, base_url="https://api.deepseek.com/v1")
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": file_content},
            {"role": "user", "content": x},
            {"role": "user", "content": 'Напиши краткий ответ в одну строчку. Только название одной компании и направлении цены, больше ничего, два слова через пробел без пояснений'}
            ]
        )
    answer = response.choices[0].message.content
    answer = answer.strip().upper()
    print(answer, end = " ")
    print()
    prediction = 0
    ticker = 0
    if "DOWN" in answer:
        prediction = -1
        answer = answer.replace("DOWN", "")
    elif "UP" in answer:
        prediction = 1
        answer = answer.replace("UP", "")
    else:
        return 0, 0
    if len(answer) < 20:
        ticker = re.sub(r'[+\-=:/]', '', answer)
        # ticker = answer  
             
    return ticker, prediction



In [8]:
df1 = df.iloc[:3,]

In [13]:
df[['ticker', 'prediction']] = df.apply(predict, axis=1, result_type='expand')


ROCKET INTERNET UP 
TCSG UP
ALRS DOWN 
$ALRS UP 
YNDX DOWN 
LUKOIL. UP 
TRNFP DOWN 
$LSRG UP 
HYDR DOWN 
YNDX UP 
NONE. 
$MOEX DOWN
$MOEX UP 
КОНТЕКСТ ОТВЕТА НА ОСНОВЕ ДАННОГО ТЕКСТА НЕВОЗМОЖЕН, ТАК КАК ТЕКСТ НЕ СОДЕРЖИТ ИНФОРМАЦИЮ О НАЗВАНИИ КОМПАНИИ И НАПРАВЛЕНИИ ЦЕНЫ. ТРЕБУЕТСЯ ПРЕДОСТАВЛЕНИЕ ДОПОЛНИТЕЛЬНОЙ ИНФОРМАЦИИ ДЛЯ ФОРМИРОВАНИЯ ОТВЕТА. 
$YNDX DOWN 
КУПИ НАЗНАЧИЛА КИТАЙСКОЕ НАРОДНОЕ УЧИЛИЩЕ ИНОСТРАННЫХ ДЕЛ СЛОВ ЧЕРЕЗ 27 НОЯБРЬ ГОДА. 
NOT APPLICABLE. 
IMPOSSIBLE TO PROVIDE A SPECIFIC ANSWER. 
$NVTK DOWN

$LKOH UP 
НЕОБХОДИМО ПОЧИТАТЬ ТЕКСТ ДЛЯ ВЫЯВЛЕНИЯ НАЗВАНИЯ КОМПАНИИ И НАПРАВЛЕНИЯ ЦЕНЫ ЦЕНЫ. 
LKOH UP 
РОСТЕЛЕКОМ UP 
TINKOFF РОССИЯ UP 
БЕЗ СВЕДЕНИЯ О НАЗВАНИИ КОМПАНИИ ИЗ ПРЕДОСТАВЛЕННОГО ТЕКСТА, НАПРАВЛЕНИЕ ЦЕНЫ ОПРЕДЕЛИТЬ НЕВОЗМОЖНО. 
$QIWI UP 
YANDEX. UP. 
THE REQUESTED INFORMATION IS NOT PROVIDED IN THE TEXT. 
N/A 
1. 2. 
ДИАСOFT DOWN 
NVTK DOWN 
NVTK DOWN 
TGKB UP

TGKBP UP

TGKN UP

TGKA UP

MAGN UP

NLMK UP 
GLTR UP 
RKKE UP 
<PRESIDIO_ANSWER> 
SFIN UP
OKEY DOWN 
GLTR 

In [18]:
df

,name,base_url,subscribers,size,month_activity,profitability,date,text,ticker,prediction
0,vadya93,https://www.tinkoff.ru/invest/social/profile/v...,94574,от 10000000,28927,246.72,2024-04-06 16:37,Мошеннические схемы портят вам торговлю в ста...,ROCKET INTERNET,1
1,vadya93,https://www.tinkoff.ru/invest/social/profile/v...,94574,от 10000000,28927,246.72,2024-04-05 06:34,Рыночек слили куда пониже на фоне...?\n\nДрузь...,TCSG UP\nALRS,-1
2,vadya93,https://www.tinkoff.ru/invest/social/profile/v...,94574,от 10000000,28927,246.72,2024-04-04 06:35,"Рынок у сопротивления, так ли все плохо?!\n\nД...",$ALRS,1
3,vadya93,https://www.tinkoff.ru/invest/social/profile/v...,94574,от 10000000,28927,246.72,2024-04-03 06:28,"Рынку выше некуда?\n\nДрузья, в общем вчера ры...",YNDX,-1
4,vadya93,https://www.tinkoff.ru/invest/social/profile/v...,94574,от 10000000,28927,246.72,2024-04-02 06:52,"Друзья, мы вчера прорвались сквозь зону продаж...",LUKOIL.,1
...,...,...,...,...,...,...,...,...,...,...
565,Karsotel,https://www.tinkoff.ru/invest/social/profile/K...,56848,до 10000000,111,258.45,2024-03-20 05:34,​РФ рынок: что нас ждет сегодня\n\nВо вторник ...,LKOH,1
566,Karsotel,https://www.tinkoff.ru/invest/social/profile/K...,56848,до 10000000,111,258.45,2024-03-19 11:09,​SFIN: закрыл в +268%\n\n\nСегодня SFI после н...,SFIN,1
567,Karsotel,https://www.tinkoff.ru/invest/social/profile/K...,56848,до 10000000,111,258.45,2024-03-19 05:13,​РФ рынок: что нас ждет сегодня\n\nВ понедельн...,FLOT,1
568,Karsotel,https://www.tinkoff.ru/invest/social/profile/K...,56848,до 10000000,111,258.45,2024-03-18 05:16,​РФ рынок: что нас ждет сегодня\n\nВ пятницу и...,TCSG,1


In [60]:
df

,name,base_url,subscribers,size,month_activity,profitability,date,text,ticker,prediction
0,vadya93,https://www.tinkoff.ru/invest/social/profile/v...,94574,от 10000000,28927,246.72,2024-04-06 16:37,Мошеннические схемы портят вам торговлю в ста...,ROCKET INTERNET,1
1,vadya93,https://www.tinkoff.ru/invest/social/profile/v...,94574,от 10000000,28927,246.72,2024-04-05 06:34,Рыночек слили куда пониже на фоне...?\n\nДрузь...,TCSG UP\nALRS,-1
2,vadya93,https://www.tinkoff.ru/invest/social/profile/v...,94574,от 10000000,28927,246.72,2024-04-04 06:35,"Рынок у сопротивления, так ли все плохо?!\n\nД...",$ALRS,1
3,vadya93,https://www.tinkoff.ru/invest/social/profile/v...,94574,от 10000000,28927,246.72,2024-04-03 06:28,"Рынку выше некуда?\n\nДрузья, в общем вчера ры...",YNDX,-1
4,vadya93,https://www.tinkoff.ru/invest/social/profile/v...,94574,от 10000000,28927,246.72,2024-04-02 06:52,"Друзья, мы вчера прорвались сквозь зону продаж...",LUKOIL.,1
...,...,...,...,...,...,...,...,...,...,...
565,Karsotel,https://www.tinkoff.ru/invest/social/profile/K...,56848,до 10000000,111,258.45,2024-03-20 05:34,​РФ рынок: что нас ждет сегодня\n\nВо вторник ...,LKOH,1
566,Karsotel,https://www.tinkoff.ru/invest/social/profile/K...,56848,до 10000000,111,258.45,2024-03-19 11:09,​SFIN: закрыл в +268%\n\n\nСегодня SFI после н...,SFIN,1
567,Karsotel,https://www.tinkoff.ru/invest/social/profile/K...,56848,до 10000000,111,258.45,2024-03-19 05:13,​РФ рынок: что нас ждет сегодня\n\nВ понедельн...,FLOT,1
568,Karsotel,https://www.tinkoff.ru/invest/social/profile/K...,56848,до 10000000,111,258.45,2024-03-18 05:16,​РФ рынок: что нас ждет сегодня\n\nВ пятницу и...,TCSG,1


In [62]:

df.to_pickle('df.pkl')



'https://www.tinkoff.ru/invest/social/profile/vadya93'

In [19]:
df1 = df.copy()

In [21]:
df1 = df1[['name', 'date', 'text', 'ticker', 'prediction']]

In [25]:
grouped_data = df1.groupby('name')

result_dict = {name: group[['date', 'text', 'ticker', 'prediction']].values.tolist() for name, group in grouped_data}


In [45]:
result_dict = {name: group[['date', 'text', 'ticker', 'prediction']].to_dict(orient='records') for name, group in grouped_data}


In [47]:
result_dict['Karsotel']

[{'date': '2024-04-10 05:29',
  'text': '\u200bРФ рынок: что нас ждет сегодня\n\nВо вторник индекс мосбиржи открылся ростом, тестировали 3450, но откатились и закрылись в нулях.\nДоллар подрос до 93, нефть корректируется и закрылась чуть ниже 90.\n\nКомпании:\n1)Positive: день инвестора\nВчера компания представила МСФО отчет за 2023.\nОтгрузки выросли на 76% г/г до 25.5 миллиардов рублей.\nNIC вырос на 76% г/г до 8.7 миллиардов рублей (чистая прибыль без учета капитализируемых расходов).\nЗаплатят 52 рубля дивиденда 2023 год.\n\nТакже провели день инвестора.\nТам дали прогноз на 2024-2025.\nВ 2024 ждут 40-50 миллиардов рублей отгрузок.\nВ 2025 ждут 70-100 миллиардов рублей отгрузок.\n\nТемпы роста поражают, хотят продолжать удваиваться. Импортозамещения играет на руку компании. Сейчас стоят 5 EV/EBITDA 2025 или 6.5 прибылей 2025. Один из самых низких показателей в секторе.\nВ портфелях продолжаю держать. \n\n2)Ozon: растущий гигант.\nОборот (GMV) вырос на 111% г/г до 1.75 триллионов ру

In [44]:
df.loc[df.name=='Karsotel', 'profitability'].values[0]

258.45

In [55]:
df.loc[df.name=='Karsotel', 'base_url'][0]

KeyError: 0

In [58]:
def save_prediction_to_json(filename, df, result_dict):
    print(df.loc[df.name=='Karsotel', 'base_url'].values[0])
    author_data_list = []
    for author in AUTHORS:
        author_data = {
            "name": author,
            "base_url": df.loc[df.name==author, 'base_url'].values[0],
            "subscribers": int(df.loc[df.name==author, 'subscribers'].values[0]),
            "size": df.loc[df.name==author, 'size'].values[0],
            "month_activity": int(df.loc[df.name==author, 'month_activity'].values[0]),
            "profitability": df.loc[df.name==author, 'profitability'].values[0],
            "articles": result_dict[author],
        }
        author_data_list.append(author_data)

    with open(filename, "w", encoding="utf-8") as json_file:
        json.dump(author_data_list, json_file, ensure_ascii=False, indent=4)

In [59]:
save_prediction_to_json('result.json',df,result_dict)

https://www.tinkoff.ru/invest/social/profile/Karsotel


In [28]:
result_dict

{'ALL_TIME_TRADING': [['2024-04-10 13:52',
   'Инфляция в США  снова растет \n\nБыло очевидно, что цен на нефть $BRM4  и сильный рынок труда будут способствовать удержанию инфляции на высоких отметках, но вряд ли участники рынка ожидали роста год к году \n\nПопытки ФРС вернуть инфляцию в стране к отметке 2% пока ни к чему не привели, а выход сегодняшних данных заставит игроков еще сильнее задуматься о более длительном периоде жесткой ДКП \n\nФьючерсы на основные фондовые индексы $SFM4 $NAM4 реагируют бурным снижением. Ожидаем аналогичную картину и на российском рынке $MMM4 \n\nПодписывайся на @ALL_TIME_TRADING ‼\n\n#Новости #новичкам #сша #обзор #аналитика #рынок #прогноз #идея #хочу_в_дайджест #нефтьЧитать дальше',
   'CISCO $CSCO ',
   1],
  ['2024-04-10 07:20',
   'Доллар $USDRUB не спас \n\nНесмотря на то, что вчера, рубль как Мы и ожидали начал девальвироваться и завершил торги у отметки 93₽, это не смогло удержать индекс Мосбиржи $MMM4 от фиксации прибыли \n\nЗато техническая кар

In [27]:
len(result_dict)

19

In [24]:
for q in df1.groupby('name'):
    print (q)

('ALL_TIME_TRADING',                  name              date  \
210  ALL_TIME_TRADING  2024-04-10 13:52   
211  ALL_TIME_TRADING  2024-04-10 07:20   
212  ALL_TIME_TRADING  2024-04-10 06:50   
213  ALL_TIME_TRADING  2024-04-09 13:55   
214  ALL_TIME_TRADING  2024-04-09 09:01   
215  ALL_TIME_TRADING  2024-04-08 18:03   
216  ALL_TIME_TRADING  2024-04-08 08:05   
217  ALL_TIME_TRADING  2024-04-08 07:46   
218  ALL_TIME_TRADING  2024-04-08 06:51   
219  ALL_TIME_TRADING  2024-04-07 20:49   
220  ALL_TIME_TRADING  2024-04-07 13:04   
221  ALL_TIME_TRADING  2024-04-06 09:38   
222  ALL_TIME_TRADING  2024-04-05 15:38   
223  ALL_TIME_TRADING  2024-04-05 10:02   
224  ALL_TIME_TRADING  2024-04-05 06:59   
225  ALL_TIME_TRADING  2024-04-04 13:00   
226  ALL_TIME_TRADING  2024-04-04 11:03   
227  ALL_TIME_TRADING  2024-04-04 08:26   
228  ALL_TIME_TRADING  2024-04-04 07:02   
229  ALL_TIME_TRADING  2024-04-03 16:25   
230  ALL_TIME_TRADING  2024-04-03 14:10   
231  ALL_TIME_TRADING  2024-04-03

In [11]:
OXYGEN_API_KEY = "oxy-UCnZ5b2l0nDkyttL2iBNJA2ylGjK3MXzuEHOY0nJpFN5R"

client = OpenAI(api_key=OXYGEN_API_KEY,base_url="https://app.oxyapi.uk/v1")

stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        
        
        
            {"role": "user", "content": file_content},
            {"role": "user", "content": df.loc[0, 'text']},
            {"role": "user", "content": 'Напиши краткий ответ в одну строчку. Только название компании и направлении цены, два слова через пробел без пояснений'}
              
              
              ],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

$MMM4 down